In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras

from tensorflow.keras.layers import Dense, Activation, AlphaDropout
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
import time
import datetime
from tensorflow.keras.callbacks import TensorBoard
import pickle

In [2]:
#file_name = "Binance_BTCUSDT_1h.csv" # 1 hour data
#file_name = "COINBASE_BTCUSD, 240.csv" # 4 hour data
file_name = "Bitfinex_BTCUSD_d\.csv" # 1 day data
data = pd.read_csv("data/2. Prepared data/"+file_name)
data.head()

,unix,open,high,low,close,X1,X2,X3_1,X3_2,X3_1_vol,...,X12,X13,X12_vol,X13_vol,X14,X15,X16,X17,X18,target
0,1424649600,236.59,241.00,232.61,239.73,0.013272,0.848629,NaN,NaN,NaN,...,-0.145911,-0.122399,187.936832,201.196495,29.861155,NaN,NaN,NaN,NaN,-0.000209
1,1424736000,239.73,240.99,236.70,239.68,-0.000209,0.694639,NaN,NaN,NaN,...,-0.123665,-0.122828,-160.104503,78.804558,29.698216,NaN,NaN,NaN,NaN,-0.007009
2,1424822400,239.68,240.83,235.53,238.00,-0.007009,0.466038,NaN,NaN,NaN,...,-0.175824,-0.138593,-473.580082,-85.517384,24.512651,NaN,NaN,NaN,NaN,-0.004202
3,1424908800,238.00,238.34,233.62,237.00,-0.004202,0.716102,NaN,NaN,NaN,...,-0.246635,-0.167879,-434.316764,-180.061409,21.887790,NaN,NaN,NaN,NaN,0.081857
4,1424995200,237.00,262.80,236.64,256.40,0.081857,0.755352,NaN,NaN,NaN,...,0.591055,0.024187,1889.619565,343.719632,77.005441,NaN,NaN,NaN,NaN,-0.002457


In [3]:
model_name = ""
if file_name == "Bitfinex_BTCUSD_d\.csv":
    model_name = "ann_1D.sav"
elif file_name == "COINBASE_BTCUSD, 240.csv":
    model_name = "ann_4H.sav"
elif file_name == "Binance_BTCUSDT_1h.csv":
    model_name = "ann_1H.sav"

In [4]:
data.shape

(2509, 39)

In [5]:
data.columns

Index(['unix', 'open', 'high', 'low', 'close', 'X1', 'X2', 'X3_1', 'X3_2',
       'X3_1_vol', 'X3_2_vol', 'X4', 'X4_vol', 'X5_1', 'X5_2', 'X5_1_vol',
       'X5_2_vol', 'X6', 'X6_vol', 'X7', 'X7_vol', 'X8', 'X9', 'X8_vol',
       'X9_vol', 'X10_1', 'X10_2', 'X11_1', 'X11_2', 'X12', 'X13', 'X12_vol',
       'X13_vol', 'X14', 'X15', 'X16', 'X17', 'X18', 'target'],
      dtype='object')

## Na values

In [6]:
data.isna().sum().sum()

1347

In [7]:
# remove na values

In [8]:
print(data.shape)
data = data.dropna()
print(data.shape)

# 1 DAY
#(2509, 39)
#(2315, 39)

# 4 Hours
#(15273, 39)
#(15086, 39)

# 1 Hour
#(40238, 39)
#(39922, 39)

(2509, 39)
(2315, 39)


## Binary target

In [9]:
data["test"] = -1
for i in range(0, len(data)):
    if data["target"].iloc[i] > 0.0025: # 1 DAY
    #if data["target"].iloc[i] > 0.00035: # 4 Hours
    #if data["target"].iloc[i] > 0.0001: # 1 Hour
        data["test"].iloc[i] = 1
    else:
        data["test"].iloc[i] = 0
    

/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_4958/4268656756.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["test"].iloc[i] = 0
/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_4958/4268656756.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["test"].iloc[i] = 1


In [14]:
#for i in range(0, len(data)):
#    print(datetime.datetime.fromtimestamp(data["unix"].iloc[i]))

In [13]:
test_split = round(len(data) * 0.9)
data_test = data[test_split:]
data = data[:test_split]

In [ ]:

data_test.to_csv("data/4. Test data for web/"+ file_name, index=False)

In [ ]:
data.corrwith(data["test"]).sort_values()

## Remove outliers

In [ ]:
def describe_data(dataset):
    values = []
    for col in dataset.columns:
        values.append({
            'Column': col,
            'Mean': round(np.mean(dataset[col]), 2),
            'Min': round(np.min(dataset[col]), 2),
            'Max': round(np.max(dataset[col]), 2),
            'Std': round(np.std(dataset[col]), 2),
            'Max Z-Val': round(np.abs(max(stats.zscore(dataset[col]))), 2)
        })
    return pd.DataFrame(values)

In [ ]:
describe_data(data)

As we can see we have a lot of outliers and we have to remove them.

In [ ]:
data.columns

In [ ]:
# Drop target from outliers removal
data_output = data[[ 'X1', 'X2', 'X3_1', 'X3_2', 'X3_1_vol', 'X3_2_vol', 'X4','X4_vol', 'X5_1',
                    'X5_2', 'X5_1_vol', 'X5_2_vol', 'X6', 'X6_vol', 'X7','X7_vol', 'X8', 'X9','X8_vol',
                    'X9_vol','X10_1','X10_2','X11_1','X11_2', 'X12', 'X13', 'X12_vol', 'X13_vol','X14', 
                    'X15', 'X16','X17', 'X18']]

In [ ]:
print(data.shape)
z_scores = stats.zscore(data_output)

abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
new_df = data[filtered_entries]
print(new_df.shape)

# 1 DAY
#(2084, 40)
#(1530, 40)

# 4 Hour
#(13577, 40)
#(10334, 40)

# 1 Hour
#(35930, 40)
#(26524, 40)

In [ ]:
describe_data(new_df)

In [ ]:
data = new_df

In [ ]:
(data.corrwith(data["test"]).abs()).sort_values(ascending=False)

## Creating X and y

In [ ]:
data.columns

In [ ]:
X = data[[ 'X1', 'X2', 'X3_1', 'X3_2', 'X3_1_vol', 'X3_2_vol', 'X4','X4_vol', 'X5_1','X5_2', 'X5_1_vol', 'X5_2_vol',
          'X6', 'X6_vol', 'X7','X7_vol', 'X8', 'X9','X8_vol','X9_vol','X10_1','X10_2','X11_1','X11_2', 'X12', 'X13', 
          'X12_vol', 'X13_vol','X14', 'X15', 'X16','X17', 'X18']].values
X

In [ ]:
y = data[["test"]].values
y

In [ ]:
print("X and y shape:")
print(X.shape)
print(y.shape)

## Data standardization

In [ ]:
scale= StandardScaler()

X = scale.fit_transform(X)
X

## Data spliting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print("Train shape:")
print(X_train.shape)
print(y_train.shape)
#print("Validation shape:")
#print(X_valid.shape)
#print(y_valid.shape)
print("Test shape:")
print(X_test.shape)
print(y_test.shape)

# Keras Tuner

In [ ]:
up = 0
down = 0
for i in range(0, len(data)):
    if data["test"].iloc[i] == 1:
        up+=1
    if data["test"].iloc[i] == 0:
        down+=1
print("All: ", len(data))
print("Up: ", up," = " ,round((up/len(data)),2),"%")
print("Down: ", down," = " ,round((down/len(data)),2),"%")

In [ ]:
from tensorflow.keras.layers import Dense, Activation, AlphaDropout
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
from keras_tuner.engine.hyperparameters import HyperParameters
import time
import datetime
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

#LOG_DIR = f"/logs/{int(time.time())}"


#tensorboard = TensorBoard(log_dir=LOG_DIR)

LOG_DIR = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + file_name
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1)


In [ ]:
def build_model(hp):
    model = keras.Sequential()
    
    model.add(Dense(units = hp.Int("input_units",2,500, 2), activation = 'relu', input_shape = (X.shape[1],)))

    dropout_rate = hp.Choice('dropout_rate', values=[0.2])#, 0.3, 0.4, 0.5, 0.6])
    model.add(AlphaDropout(dropout_rate))
    
    
    for i in range(hp.Int('n_layers', 2, 7)):  # adding variation of layers.
        model.add(Dense(hp.Int(f'hidden_{i}_unit',
                                2,500, 2), activation = 'relu'))
    
        model.add(AlphaDropout(dropout_rate))


    model.add(Dense(units = 1, activation= 'sigmoid'))
    
    #p_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    initial_learning_rate = 0.01
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=100000,
        decay_rate=0.96,
        staircase=True)
    
    model.compile(
    loss=tf.keras.losses.mae, 
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
    metrics=['accuracy'])
    
    return model
LOG_DIR = f"{int(time.time())}"

In [ ]:
tuner = BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=100,
    #executions_per_trial=2,
    directory=LOG_DIR
)

In [ ]:
tuner.search(
    x= X_train, 
    y=y_train, 
    epochs=100, 
    batch_size=64, 
    #validation_data=(X_valid, y_valid),
    validation_split=0.2,
    callbacks= [EarlyStopping(monitor='accuracy', patience=20)]
)

# 1 DAY
# Best val_accuracy So Far: 0.6326531171798706

# 4 Hours
# Best val_accuracy So Far: 0.568923830986023

# 1 Hour
# Best val_accuracy So Far: 0.5581998229026794

In [ ]:
best_model = tuner.get_best_models()[0]

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
best_model.summary()

In [ ]:
best_model.evaluate(X_test, y_test)

In [ ]:
predictions = best_model.predict(X_test)
predictions

In [ ]:
tuner.results_summary()

In [ ]:
#%load_ext tensorboard

In [ ]:
#%tensorboard --logdir logs

In [ ]:
pickle.dump(best_model, open("models/"+model_name, 'wb'))

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
loaded_model = pickle.load(open("models/"+model_name, 'rb'))
result = loaded_model.evaluate(X_valid, y_valid)
print(result)

# 1 DAY
# [0.35910338163375854, 0.644897997379303]

# 4 Hours
# [0.4530210494995117, 0.547158420085907]

# 1 Hour
# [0.44210100173950195, 0.5577285289764404]

In [ ]:
loaded_model.summary()